In [1]:
import os
import pickle
import collections
import datetime
import copy
import numpy as np
import scipy as sp
import pandas as pd
import happybase
import warnings
import re
import csv
import json
import tensorflow as tf
import sklearn.model_selection
import sklearn.preprocessing

In [2]:
data_dir = r"/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data"

In [3]:
file_list = []
for root,_,path_list in os.walk(data_dir):
    for path in path_list:
        file_list.append(os.path.join(root,path))
print(file_list)

['/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000000_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000002_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000001_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000003_0']


In [4]:
%%time
#读取用户数据
all_file_data = []
for file_name in file_list:
    all_file_data.append(pd.read_csv(os.path.join(data_dir,file_name),sep='\001',header=None,error_bad_lines=False))
df = pd.concat(all_file_data,axis=0,ignore_index=True)
del all_file_data
column_list = ['用户id','用户姓名','用户性别','用户出生时间','所在公司','所处职位','行业','微信网名','微信性别','常驻省','常驻市','是否付费用户','付费等级','关注的作者','作者类型','关注的马甲','关注的公司','文章id','是否精选文章','浏览文章时间','浏览时长','文章频道','文章类型','文章标签','文章图片数量','文章字数','新定位省','新定位市','评论内容','打赏金额','设备型号','使用网络']
df.columns = column_list
old_df = copy.deepcopy(df)

b'Skipping line 461777: expected 32 fields, saw 33\n'
<string>:2: DtypeWarning: Columns (1,3,4,5,6,7,15,16,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
<string>:2: DtypeWarning: Columns (1,2,3,4,5,6,7,15,16,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 57.1 s, sys: 3.46 s, total: 1min
Wall time: 1min 6s


In [5]:
use_column_list = ["用户id",'行业',"所在公司","关注的公司"]
y_col = '行业'

In [6]:
df = copy.deepcopy(old_df)
df

,用户id,用户姓名,用户性别,用户出生时间,所在公司,所处职位,行业,微信网名,微信性别,常驻省,...,文章类型,文章标签,文章图片数量,文章字数,新定位省,新定位市,评论内容,打赏金额,设备型号,使用网络
0,00001D26FE991A4BC4A01325D5E222DE,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,"年轻人,-1",-1,"-1,2","-1,1274","重庆,四川","重庆,成都",-1,-1,LIO-AN00,"移动网络,WIFI"
1,00006C3EF7F79B0F37EEFB8381505530,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,"-1,股市,社会,医疗,文娱其它,手机,爱奇艺",-1,"13,1,5,7,6","111,9529,6301,6733,2332,10024,2425,11986,6141",北京,北京,-1,-1,BLA-AL00,"移动网络,WIFI"
2,0000D0E1960239E8362F6B4100D9790A,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,陕西,西安,-1,-1,Redmi K20 Pro,移动网络
3,0000f55e60d5ba7b36004429b235c95096f7153d,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,广东,"佛山,广州",-1,-1,iPhone 11 Pro,移动网络
4,00014E9D5CF948B3FE6376564B23F64F,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,山西,太原,-1,-1,ANE-AL00,移动网络
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15703260,qlu2xsthqk7tp0mncpgr,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,8,5996,北京,北京,-1,-1,-1,未知
15703261,qlu30bysbp1mj150uc1m,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,11,6625,未知,未知,-1,-1,-1,未知
15703262,qlu30t7n6f2b1jgzexv,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,未知,未知,-1,-1,-1,未知
15703263,qlu315b37v5rc273amb0,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,空,"昆仑万维,陈一舟,游戏公司,股权,昆仑游戏,周亚辉",3,4416,未知,未知,-1,-1,-1,未知


In [7]:
df = df[use_column_list]
df

,用户id,行业,所在公司,关注的公司
0,00001D26FE991A4BC4A01325D5E222DE,-1,-1,-1
1,00006C3EF7F79B0F37EEFB8381505530,-1,-1,-1
2,0000D0E1960239E8362F6B4100D9790A,-1,-1,-1
3,0000f55e60d5ba7b36004429b235c95096f7153d,-1,-1,-1
4,00014E9D5CF948B3FE6376564B23F64F,-1,-1,-1
...,...,...,...,...
15703260,qlu2xsthqk7tp0mncpgr,-1,-1,-1
15703261,qlu30bysbp1mj150uc1m,-1,-1,-1
15703262,qlu30t7n6f2b1jgzexv,-1,-1,-1
15703263,qlu315b37v5rc273amb0,-1,-1,-1


# 数据分析

In [44]:
df["行业"].unique()

array(['-1', '内容/营销/传播', -1, '教育/专业服务/培训', '创业者', '互联网', '电商/仓储物流', '投资人',
       'IT', 'TMT从业人员', '商业服务', '酒店/餐饮/旅游', '金融', '政府/非盈利机构',
       '企业服务/云计算/大数据', '房地产', '汽车/出行', '媒体', '加工制造', '其他', '制药医疗/生物/卫生保健',
       '文化/体育/娱乐业', '商业服务-O2O/服务/社区', '人工智能/前沿科技', '法律', '智能硬件/硬件制造', '0',
       '贸易/零售批发', '学生', '能源相关', 'undefined', '农业', '仓储物流', '零售/快消', 0,
       '营销'], dtype=object)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15703265 entries, 0 to 15703264
Data columns (total 6 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   行业      object
 1   关注的公司   object
 2   新定位省    object
 3   新定位市    object
 4   文章标签    object
 5   文章类型    object
dtypes: object(6)
memory usage: 718.8+ MB


# 模型训练

In [8]:
df.fillna("",inplace=True)
#行业列
df[y_col] = df[y_col].apply(lambda x : str(x))
df

/home/zhaodachuan/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/home/zhaodachuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,用户id,行业,所在公司,关注的公司
0,00001D26FE991A4BC4A01325D5E222DE,-1,-1,-1
1,00006C3EF7F79B0F37EEFB8381505530,-1,-1,-1
2,0000D0E1960239E8362F6B4100D9790A,-1,-1,-1
3,0000f55e60d5ba7b36004429b235c95096f7153d,-1,-1,-1
4,00014E9D5CF948B3FE6376564B23F64F,-1,-1,-1
...,...,...,...,...
15703260,qlu2xsthqk7tp0mncpgr,-1,-1,-1
15703261,qlu30bysbp1mj150uc1m,-1,-1,-1
15703262,qlu30t7n6f2b1jgzexv,-1,-1,-1
15703263,qlu315b37v5rc273amb0,-1,-1,-1


In [9]:
df[y_col].unique()

array(['-1', '内容/营销/传播', '教育/专业服务/培训', '创业者', '互联网', '电商/仓储物流', '投资人',
       'IT', 'TMT从业人员', '商业服务', '酒店/餐饮/旅游', '金融', '政府/非盈利机构',
       '企业服务/云计算/大数据', '房地产', '汽车/出行', '媒体', '加工制造', '其他', '制药医疗/生物/卫生保健',
       '文化/体育/娱乐业', '商业服务-O2O/服务/社区', '人工智能/前沿科技', '法律', '智能硬件/硬件制造', '0',
       '贸易/零售批发', '学生', '能源相关', 'undefined', '农业', '仓储物流', '零售/快消', '营销'],
      dtype=object)

In [10]:
def deal_with_trade(trade):
    if trade in set(["-1","0","undefined"]):
        return False
    return True

predict_df = df[-df[y_col].apply(deal_with_trade)]
train_df = df[df[y_col].apply(deal_with_trade)]

In [11]:
train_df

,用户id,行业,所在公司,关注的公司
13218,10113,内容/营销/传播,-1,-1
38003,109934,教育/专业服务/培训,-1,-1
41738,11118,创业者,-1,-1
41739,11118,创业者,-1,-1
41740,11118,创业者,-1,-1
...,...,...,...,...
14475059,77942,互联网,-1,-1
14475060,77942,互联网,-1,-1
14475061,77942,互联网,-1,-1
14501459,84249,互联网,-1,-1


In [12]:
train_df["行业"].unique()

array(['内容/营销/传播', '教育/专业服务/培训', '创业者', '互联网', '电商/仓储物流', '投资人', 'IT',
       'TMT从业人员', '商业服务', '酒店/餐饮/旅游', '金融', '政府/非盈利机构', '企业服务/云计算/大数据',
       '房地产', '汽车/出行', '媒体', '加工制造', '其他', '制药医疗/生物/卫生保健', '文化/体育/娱乐业',
       '商业服务-O2O/服务/社区', '人工智能/前沿科技', '法律', '智能硬件/硬件制造', '贸易/零售批发', '学生',
       '能源相关', '农业', '仓储物流', '零售/快消', '营销'], dtype=object)

In [13]:
predict_df

,用户id,行业,所在公司,关注的公司
0,00001D26FE991A4BC4A01325D5E222DE,-1,-1,-1
1,00006C3EF7F79B0F37EEFB8381505530,-1,-1,-1
2,0000D0E1960239E8362F6B4100D9790A,-1,-1,-1
3,0000f55e60d5ba7b36004429b235c95096f7153d,-1,-1,-1
4,00014E9D5CF948B3FE6376564B23F64F,-1,-1,-1
...,...,...,...,...
15703260,qlu2xsthqk7tp0mncpgr,-1,-1,-1
15703261,qlu30bysbp1mj150uc1m,-1,-1,-1
15703262,qlu30t7n6f2b1jgzexv,-1,-1,-1
15703263,qlu315b37v5rc273amb0,-1,-1,-1


In [14]:
replace_dict = {
    '内容/营销/传播':"其他",  
    '教育/专业服务/培训':"其他", 
    '创业者':"其他", 
    '互联网':"IT/移动互联网", 
    '电商/仓储物流':"电商", 
    '投资人':"金融",
    'IT':"IT/移动互联网", 
    'TMT从业人员':"IT/移动互联网", 
    '商业服务':"其他", 
    '酒店/餐饮/旅游':"其他", 
    '金融':"金融", 
    '政府/非盈利机构':"其他",
    '企业服务/云计算/大数据':"IT/移动互联网", 
    '房地产':"房地产", 
    '汽车/出行':"汽车", 
    '媒体':"其他", 
    '加工制造':"制造业", 
    '其他':"其他", 
    '制药医疗/生物/卫生保健':"医疗",
    '文化/体育/娱乐业':"游戏", 
    '商业服务-O2O/服务/社区':"其他",
    '人工智能/前沿科技':"IT/移动互联网",
    '法律':"其他", 
    '智能硬件/硬件制造':"IOT", 
    '贸易/零售批发':"零售消费", 
    '学生':"游戏", 
    '能源相关':"其他", 
    '农业':"其他", 
    '仓储物流':"其他", 
    '零售/快消':"零售消费",
    '营销':"零售消费"
}
train_df[y_col].replace(replace_dict,inplace=True)
train_df[y_col].unique()

/home/zhaodachuan/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


array(['其他', 'IT/移动互联网', '电商', '金融', '房地产', '汽车', '制造业', '医疗', '游戏',
       'IOT', '零售消费'], dtype=object)

In [15]:
y_onehot = sklearn.preprocessing.LabelBinarizer()
y_onehot.fit(train_df[y_col].unique())

def bucket_trade_ont_hot(gender_str,y_onehot):
    return y_onehot.transform([gender_str])[0]

train_df[y_col] = train_df[y_col].apply(lambda x:bucket_trade_ont_hot(x,y_onehot))

/home/zhaodachuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [16]:
train_df

,用户id,行业,所在公司,关注的公司
13218,10113,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",-1,-1
38003,109934,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",-1,-1
41738,11118,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",-1,-1
41739,11118,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",-1,-1
41740,11118,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",-1,-1
...,...,...,...,...
14475059,77942,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",-1,-1
14475060,77942,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",-1,-1
14475061,77942,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",-1,-1
14501459,84249,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",-1,-1


In [17]:
#训练数据
multi_hot_column_odict = collections.OrderedDict()
multi_hot_column_set = set([v for v in use_column_list if v!="用户id"])
for i,column in enumerate(multi_hot_column_set):
    print(column)
    if column == y_col:
        continue
    multi_hot_column_odict[column] = sklearn.preprocessing.MultiLabelBinarizer()
    split_column_list = [item.split(",") if isinstance(item,str) else str(item) for item in train_df[column] ]
    #multi_hot_column_odict[column].fit(split_column_list)
    train_df[column] = multi_hot_column_odict[column].fit_transform(split_column_list).tolist()

所在公司


/home/zhaodachuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


关注的公司
行业


In [18]:
for k,v in multi_hot_column_odict.items():
    print(k,len(v.classes_))

所在公司 476
关注的公司 249


In [19]:
train, test = sklearn.model_selection.train_test_split(train_df, test_size=0.2)
train, val = sklearn.model_selection.train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

105073 train examples
26269 validation examples
32836 test examples


In [20]:
def generate_xy(data_df, y_col, multi_hot_column_odict, shuffle=True):
    """"""
    x_df = data_df.copy()
    x_list = []
    y_list = []
    for i,index in enumerate(x_df.index):
        temp_x_list = np.concatenate(x_df.loc[index,[column for column in multi_hot_column_odict]]).tolist()
        temp_y_list = x_df.loc[index,y_col]
        x_list.append(temp_x_list)
        y_list.append(temp_y_list)
        if i % 10000 == 0:
            print(i)
    return np.array(x_list,dtype=np.float32), np.array(y_list,dtype=np.int32)

In [21]:
%%time
x_train,y_train = generate_xy(train,y_col,multi_hot_column_odict)
x_valid,y_valid = generate_xy(val,y_col,multi_hot_column_odict)
x_test,y_test = generate_xy(test,y_col,multi_hot_column_odict)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
0
10000
20000
0
10000
20000
30000
CPU times: user 1min 48s, sys: 2.14 s, total: 1min 50s
Wall time: 1min 41s


In [22]:
input_dim = 0
for k,v in  multi_hot_column_odict.items():
    input_dim += len(v.classes_)
input_dim

725

In [23]:
y_dim = len(y_onehot.classes_)
y_dim

11

In [24]:
with tf.device('/device:GPU:1'):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(input_dim,)))
    model.add(tf.keras.layers.Dense(y_dim, activation='softmax',use_bias=True,
                                    #kernel_regularizer=tf.keras.regularizers.l1(0.01)
                                   ))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=[tf.keras.metrics.CategoricalAccuracy()])
    model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 11)                7986      
Total params: 7,986
Trainable params: 7,986
Non-trainable params: 0
_________________________________________________________________


In [25]:
x_train.sum

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [26]:
collections.Counter([str(v) for v in y_train])

Counter({'[0 0 1 0 0 0 0 0 0 0 0]': 29610,
         '[0 1 0 0 0 0 0 0 0 0 0]': 57415,
         '[0 0 0 0 0 0 0 0 1 0 0]': 2177,
         '[0 0 0 0 0 0 0 1 0 0 0]': 2617,
         '[1 0 0 0 0 0 0 0 0 0 0]': 543,
         '[0 0 0 0 0 0 0 0 0 1 0]': 8467,
         '[0 0 0 0 0 0 1 0 0 0 0]': 276,
         '[0 0 0 0 0 1 0 0 0 0 0]': 2650,
         '[0 0 0 1 0 0 0 0 0 0 0]': 669,
         '[0 0 0 0 1 0 0 0 0 0 0]': 640,
         '[0 0 0 0 0 0 0 0 0 0 1]': 9})

In [29]:
model.fit(x_train, 
          y_train, 
          batch_size=64,
          epochs=50,
         validation_data = (x_valid,y_valid))

Epoch 1/50
1642/1642 [==============================] - 2s 1ms/step - loss: 0.9079 - categorical_accuracy: 0.6544 - val_loss: 0.9161 - val_categorical_accuracy: 0.6520
Epoch 2/50
1642/1642 [==============================] - 2s 1ms/step - loss: 0.9075 - categorical_accuracy: 0.6546 - val_loss: 0.9160 - val_categorical_accuracy: 0.6522
Epoch 3/50
1642/1642 [==============================] - 2s 1ms/step - loss: 0.9072 - categorical_accuracy: 0.6548 - val_loss: 0.9158 - val_categorical_accuracy: 0.6521
Epoch 4/50
1642/1642 [==============================] - 2s 1ms/step - loss: 0.9069 - categorical_accuracy: 0.6545 - val_loss: 0.9158 - val_categorical_accuracy: 0.6523
Epoch 5/50
1642/1642 [==============================] - 2s 1ms/step - loss: 0.9067 - categorical_accuracy: 0.6548 - val_loss: 0.9158 - val_categorical_accuracy: 0.6529
Epoch 6/50
1642/1642 [==============================] - 2s 1ms/step - loss: 0.9064 - categorical_accuracy: 0.6547 - val_loss: 0.9156 - val_categorical_accuracy:

In [30]:
model.evaluate(x_test,y_test)

1027/1027 [==============================] - 1s 828us/step - loss: 0.9076 - categorical_accuracy: 0.6574


[0.9075847268104553, 0.6574491262435913]

In [33]:
predict_df

,用户id,行业,所在公司,关注的公司
0,00001D26FE991A4BC4A01325D5E222DE,-1,-1,-1
1,00006C3EF7F79B0F37EEFB8381505530,-1,-1,-1
2,0000D0E1960239E8362F6B4100D9790A,-1,-1,-1
3,0000f55e60d5ba7b36004429b235c95096f7153d,-1,-1,-1
4,00014E9D5CF948B3FE6376564B23F64F,-1,-1,-1
...,...,...,...,...
15703260,qlu2xsthqk7tp0mncpgr,-1,-1,-1
15703261,qlu30bysbp1mj150uc1m,-1,-1,-1
15703262,qlu30t7n6f2b1jgzexv,-1,-1,-1
15703263,qlu315b37v5rc273amb0,-1,-1,-1


In [35]:
import happybase
pool = happybase.ConnectionPool(size=100, host="47.110.125.69", port=9090, timeout=None,
                                protocol='binary'
                                )
with pool.connection() as conn:
    table = conn.table("feature_center:user_profile")
    with table.batch(batch_size=400) as batch:
        for i,index in enumerate(train_df.index):
            batch.put(str(train_df.loc[index,"用户id"]).encode(), {"attribute:input_trade".encode(): y_onehot.inverse_transform(np.array([train_df.loc[index,y_col]]))[0].encode()})
            #print(str(train_df.loc[index,"用户id"]).encode(),{"attribute:input_trade".encode(): y_onehot.inverse_transform(np.array([train_df.loc[index,y_col]]))[0].encode()})
            #break
            #batch.delete(str(train_df.loc[index,y_col]).encode())
            

In [36]:
#保存模型
with open("multi_hot_column_odict.pkl","wb") as f:
    pickle.dump(multi_hot_column_odict,f)
with open("y_onehot.pkl","wb") as f:
    pickle.dump(y_onehot,f)
model.save("model_output")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model_output/assets
